In [1]:
import numpy as np
import random
import scipy.misc as sp

from q1_softmax import softmax
from q2_gradcheck import gradcheck_naive
from q2_sigmoid import sigmoid, sigmoid_grad

def normalizeRows(x):
    """ Row normalization function """
    # Implement a function that normalizes each row of a matrix to have unit length
    
    ### YOUR CODE HERE
    row_norm = np.sqrt(np.sum(x * x, axis=1, keepdims=True))
    x = x / row_norm
    ### END YOUR CODE
    
    return x

def test_normalize_rows():
    print "Testing normalizeRows..."
    x = normalizeRows(np.array([[3.0,4.0],[1, 2]])) 
    # the result should be [[0.6, 0.8], [0.4472, 0.8944]]
    print x
    assert (x.all() == np.array([[0.6, 0.8], [0.4472, 0.8944]]).all())
    print "Ok normalizeRows !"


In [2]:
def gradcheck_naive(f, x):
    """ 
    Gradient check for a function f 
    - f should be a function that takes a single argument and outputs the cost and its gradients
    - x is the point (numpy array) to check the gradient at
    """ 

    rndstate = random.getstate()
    random.setstate(rndstate)  
    fx, grad = f(x) # Evaluate function value at original point
    h = 1e-4

    # Iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index

        ### try modifying x[ix] with h defined above to compute numerical gradients
        ### make sure you call random.setstate(rndstate) before calling f(x) each time, this will make it 
        ### possible to test cost functions with built in randomness later
        ### YOUR CODE HERE:
        epsilon = np.zeros(x.shape)
        epsilon[ix] = h

        random.setstate(rndstate)
        f1 = f(x + epsilon)

        random.setstate(rndstate)
        f2 = f(x - epsilon)
        
        numgrad = (f1[0] - f2[0])/2/h
        ### END YOUR CODE

        # Compare gradients
        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
        if reldiff > 1e-5:
            print "Gradient check failed."
            print "First gradient error found at index %s" % str(ix)
            print "Your gradient: %f \t Numerical gradient: %f" % (grad[ix], numgrad)
            return
    
        it.iternext() # Step to next dimension

    print "Gradient check passed!"

In [12]:
def word2vec_sgd_wrapper(word2vecModel, tokens, wordVectors, dataset, C, word2vecCostAndGradient = softmaxCostAndGradient):
    batchsize = 50
    cost = 0.0
    grad = np.zeros(wordVectors.shape)
    N = wordVectors.shape[0]
    inputVectors = wordVectors[:N/2,:]
    outputVectors = wordVectors[N/2:,:]
    for i in xrange(batchsize):
        C1 = random.randint(1,C)
        centerword, context = dataset.getRandomContext(C1)
        
        if word2vecModel == skipgram:
            denom = 1
        else:
            denom = 1
        
        c, gin, gout = word2vecModel(centerword, C1, context, tokens, inputVectors, outputVectors, dataset, word2vecCostAndGradient)
        cost += c / batchsize / denom
        grad[:N/2, :] += gin / batchsize / denom
        grad[N/2:, :] += gout / batchsize / denom
        
    return cost, grad


In [13]:
def test_word2vec():
    # Interface to the dataset for negative sampling
    dataset = type('dummy', (), {})()
    def dummySampleTokenIdx():
        return random.randint(0, 4)

    def getRandomContext(C):
        tokens = ["a", "b", "c", "d", "e"]
        return tokens[random.randint(0,4)], [tokens[random.randint(0,4)] \
           for i in xrange(2*C)]
    dataset.sampleTokenIdx = dummySampleTokenIdx
    dataset.getRandomContext = getRandomContext

    random.seed(31415)
    np.random.seed(9265)
    dummy_vectors = normalizeRows(np.random.randn(10,3))
    dummy_tokens = dict([("a",0), ("b",1), ("c",2),("d",3),("e",4)])
    print "==== Gradient check for skip-gram ===="
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(skipgram, dummy_tokens, vec, dataset, 5), dummy_vectors)
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(skipgram, dummy_tokens, vec, dataset, 5, negSamplingCostAndGradient), dummy_vectors)
    print "\n==== Gradient check for CBOW      ===="
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(cbow, dummy_tokens, vec, dataset, 5), dummy_vectors)
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(cbow, dummy_tokens, vec, dataset, 5, negSamplingCostAndGradient), dummy_vectors)

    print "\n=== Results ==="
    print skipgram("c", 3, ["a", "b", "e", "d", "b", "c"], dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset)
    print skipgram("c", 1, ["a", "b"], dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset, negSamplingCostAndGradient)
    print cbow("a", 2, ["a", "b", "c", "a"], dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset)
    print cbow("a", 2, ["a", "b", "a", "c"], dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset, negSamplingCostAndGradient)


In [14]:
    # Interface to the dataset for negative sampling
    dataset = type('dummy', (), {})()
    def dummySampleTokenIdx():
        return random.randint(0, 4)

    def getRandomContext(C):
        tokens = ["a", "b", "c", "d", "e"]
        return tokens[random.randint(0,4)], [tokens[random.randint(0,4)] \
           for i in xrange(2*C)]
    dataset.sampleTokenIdx = dummySampleTokenIdx
    dataset.getRandomContext = getRandomContext

    random.seed(31415)
    np.random.seed(9265)
    dummy_vectors = normalizeRows(np.random.randn(10,3))
    dummy_tokens = dict([("a",0), ("b",1), ("c",2),("d",3),("e",4)])


In [15]:
print "hello: " + str(dummy_tokens)

hello: {'a': 0, 'c': 2, 'b': 1, 'e': 4, 'd': 3}


In [16]:
def softmaxCostAndGradient(predicted, target, outputVectors, dataset):
    """ Softmax cost function for word2vec models """
    
    # Implement the cost and gradients for one predicted word vector  
    # and one target word vector as a building block for word2vec     
    # models, assuming the softmax prediction function and cross      
    # entropy loss.                                                   
    
    # Inputs:                                                         
    # - predicted: numpy ndarray, predicted word vector (\hat{v} in 
    #   the written component or \hat{r} in an earlier version)
    # - target: integer, the index of the target word               
    # - outputVectors: "output" vectors (as rows) for all tokens     
    # - dataset: needed for negative sampling, unused here.         
    
    # Outputs:                                                        
    # - cost: cross entropy cost for the softmax word prediction    
    # - gradPred: the gradient with respect to the predicted word   
    #        vector                                                
    # - grad: the gradient with respect to all the other word        
    #        vectors                                               
    
    # We will not provide starter code for this function, but feel    
    # free to reference the code you previously wrote for this        
    # assignment!                                                  
    
    ### YOUR CODE HERE
    
    # Compute softmax matrix
    prod = np.matmul(outputVectors, predicted)
    softmax_matrix = softmax(prod)
    #sumexp = np.sum(np.exp(prod)) => overflow

    # Compute cost
    cost = - np.log(softmax_matrix)[target]

 	# Compute gradients
    negpart = np.matmul(softmax_matrix, outputVectors)
#    print "softmax_matrix: " + str(softmax_matrix)
#    print "softmax_matrix shape: " + str(softmax_matrix.shape)
#    print "outputVectors: " + str(outputVectors)
#    print "outputVectors shape: " + str(outputVectors.shape)
#    print "negpart: " + str(negpart)
    gradPred = - outputVectors[target] + negpart

    grad = np.outer(softmax_matrix, predicted)
    grad[target] -= predicted

    ### END YOUR CODE
    
    return cost, gradPred, grad

In [50]:
def negSamplingCostAndGradient(predicted, target, outputVectors, dataset, 
    K=10):
    """ Negative sampling cost function for word2vec models """

    # Implement the cost and gradients for one predicted word vector  
    # and one target word vector as a building block for word2vec     
    # models, using the negative sampling technique. K is the sample  
    # size. You might want to use dataset.sampleTokenIdx() to sample  
    # a random word index. 
    # 
    # Note: See test_word2vec below for dataset's initialization.
    #                                       
    # Input/Output Specifications: same as softmaxCostAndGradient     
    # We will not provide starter code for this function, but feel    
    # free to reference the code you previously wrote for this        
    # assignment!
    
    ### YOUR CODE HERE
#    print "predicted: " + str(predicted)
#    print "target: " + str(target)
#    print "outputVectors" + str(outputVectors)
    # Sample K negative samples
    neg_samples = np.zeros((K,predicted.size))
    neg_sample_idx = []
    for k in range(K):
        idx = dataset.sampleTokenIdx()
        while idx == target:
            idx = dataset.sampleTokenIdx()
        neg_sample_idx.append(idx)
    	neg_samples[k] = outputVectors[idx]
#    print "neg_sample_idx" + str(neg_sample_idx)
#    print "neg_samples: "+ str(neg_samples)

    prod = np.matmul(neg_samples, predicted)
#    print "prod: " + str(prod)
    # Compute cost
    cost = - np.log(sigmoid(np.dot(outputVectors[target], predicted))) - np.sum(np.log(sigmoid(-prod)))
#    print "cost: " + str(cost)

 	# Compute gradients
    sigm = sigmoid(-np.dot(outputVectors[target], predicted))
    
    negpart = np.zeros(predicted.shape)
    for k in range(10):
        negpart += sigmoid(prod)[k] * neg_samples[k]
#    print "negative samples:" + str(neg_samples)
#    print "negative samples shape:" + str(neg_samples.shape)
#    print "sigmoid(prod): " + str(sigmoid(prod))
#    print "sigmoid(prod) shape: " + str(sigmoid(prod).shape)
#    print "negmat: " + str(negmat)
#    print "negmat shape: " + str(negmat.shape)
#	negpart = np.sum(negmat, axis = 0)
#    print "negpart: " + str(negpart)
    gradPred = - sigm * outputVectors[target] + negpart
#    print "gradPred: " + str(gradPred)

    grad = np.zeros(outputVectors.shape)
    for i in range(K):
        grad[neg_sample_idx[i]] += sigmoid(prod[i]) * predicted
    grad[target] -= sigm * predicted
#    print "grad: " + str(grad)

    ### END YOUR CODE
    
    return cost, gradPred, grad

In [51]:
def skipgram(currentWord, C, contextWords, tokens, inputVectors, outputVectors, 
    dataset, word2vecCostAndGradient = softmaxCostAndGradient):
    """ Skip-gram model in word2vec """

    # Implement the skip-gram model in this function.

    # Inputs:                                                         
    # - currrentWord: a string of the current center word           
    # - C: integer, context size                                    
    # - contextWords: list of no more than 2*C strings, the context words                                               
    # - tokens: a dictionary that maps words to their indices in    
    #      the word vector list                                
    # - inputVectors: "input" word vectors (as rows) for all tokens           
    # - outputVectors: "output" word vectors (as rows) for all tokens         
    # - word2vecCostAndGradient: the cost and gradient function for 
    #      a prediction vector given the target word vectors,  
    #      could be one of the two cost functions you          
    #      implemented above

    # Outputs:                                                        
    # - cost: the cost function value for the skip-gram model       
    # - grad: the gradient with respect to the word vectors         
    # We will not provide starter code for this function, but feel    
    # free to reference the code you previously wrote for this        
    # assignment!

    ### YOUR CODE HERE
    pred_index = tokens[currentWord]
    cost = 0
    gradIn = np.zeros(inputVectors.shape)
    gradOut = np.zeros(outputVectors.shape)
    for m in range(2*C):
    	target_index = tokens[contextWords[m]]
    	single_cost, gradPred, grad = word2vecCostAndGradient(inputVectors[pred_index], target_index, outputVectors, dataset)
#        print "cost: " + str(single_cost)
#        print "gradPred: " + str(gradPred)
#        print "grad: " + str(grad)
    	cost += single_cost
    	gradIn[pred_index] += gradPred
    	gradOut += grad
    ### END YOUR CODE
    
    return cost, gradIn, gradOut


In [52]:
#word2vec_sgd_wrapper(skipgram, dummy_tokens, dummy_vectors, dataset, 5)
#word2vec_sgd_wrapper(word2vecModel, tokens, wordVectors, dataset, C, word2vecCostAndGradient = softmaxCostAndGradient)

batchsize = 50
cost = 0.0
grad = np.zeros(dummy_vectors.shape)
N = dummy_vectors.shape[0]
inputVectors = dummy_vectors[:N/2,:]
outputVectors = dummy_vectors[N/2:,:]
C1 = random.randint(1,5)
centerword, context = dataset.getRandomContext(C1)
c, gin, gout = skipgram(centerword, C1, context, dummy_tokens, inputVectors, outputVectors, dataset, negSamplingCostAndGradient)
print "c: " +str(c)
print "gin: " + str(gin)
print"gout: "+str(gout)

c: 27.1730762768
gin: [[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-5.78657829  1.02888708 -3.68467947]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
gout: [[-1.07736921  0.49437484  3.76059273]
 [-1.10088306  0.50516469  3.84266862]
 [-0.97879559  0.44914214  3.41651825]
 [-1.15876239  0.53172391  4.04469832]
 [-0.09848948  0.04519409  0.34378076]]


In [45]:
predicted = np.array([-0.56713774, -0.27178229, -0.77748902])
neg_samples= np.array([[ 0.18289107,  0.76098587, -0.62245591],
 [-0.6831809,  -0.04200519,  0.72904007],
 [-0.6831809,  -0.04200519,  0.72904007],
 [ 0.18289107,  0.76098587, -0.62245591],
 [-0.61517874,  0.5147624,  -0.59713884],
 [-0.52629529, -0.78190408,  0.33412466],
 [-0.61517874,  0.5147624,  -0.59713884],
 [ 0.18289107,  0.76098587, -0.62245591],
 [ 0.18289107,  0.76098587, -0.62245591],
 [-0.6831809,  -0.04200519,  0.72904007]])
print "-3: "+str(predicted.shape)
print "-2: "+str(neg_samples.shape)
prod = np.matmul(neg_samples, predicted)
print "-1: "+str(prod.shape)
negmat = np.matmul(neg_samples.T,sigmoid(prod))
print "0: "+str(negmat.shape)
print "1: "+str(negmat[0])
vec_sum = np.zeros(predicted.shape)
for k in range(10):
    vec_sum += sigmoid(prod)[k] * neg_samples[k]
print "2: "+str(vec_sum)

-3: (3,)
-2: (10, 3)
-1: (10,)
0: (3,)
1: -1.65231213606
2: [-1.65231214  1.83785522 -0.9535864 ]


In [53]:
test_word2vec()

==== Gradient check for skip-gram ====
Gradient check passed!
Gradient check passed!

==== Gradient check for CBOW      ====


NameError: global name 'cbow' is not defined